In [1]:
%cd ~/austin_big_vision
import jax
import jax.numpy as jnp
import ml_collections

/mnt/vlm-pd/miniconda3/envs/vlm/lib/python3.11/site-packages/IPython/core/magics/osm.py:393: UserWarning: using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})
/mnt/vlm-pd/miniconda3/envs/vlm/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/home/austinwang/austin_big_vision


# ViT

In [ ]:
import big_vision.models.vit as model_mod
model_cfg = ml_collections.ConfigDict()
model_cfg.num_classes = None
model_cfg.patch_size = (16, 16)
model_cfg.width = 768
model_cfg.depth = 12
model_cfg.mlp_dim = None  # Defaults to 4x input dim
model_cfg.num_heads = 12
model_cfg.posemb = "learn"  # Can also be "sincos2d"
model_cfg.rep_size = False
model_cfg.dropout = 0.0
model_cfg.pool_type = "map"  # Can also be "map" or "tok"
model_cfg.head_zeroinit = True
model_cfg.mask = None
model_cfg.normalize_qk = False
model_cfg.scan = True
model_cfg.remat_policy = "nothing_saveable"
model_cfg.dtype_mm = "bfloat16"

model = model_mod.Model(**model_cfg)

VARIANT, RES = 'B/16', 224
CKPT = 'gs://us-central2-storage/tensorflow_datasets/cappa_bs16384_warm0.02_lr1e-3_wd1e-4_bf16_b2-0.95_6lyr_06-15_2102/checkpoint.bv-000183105/'
TXTVARIANT, EMBDIM, SEQLEN, VOCAB = 'B', 768, 64, 32_000

init_shapes, init_types = [(1, RES, RES, 3), (1, SEQLEN,)],['bfloat16', 'int32']
init_params = [
    jnp.zeros(shape, dtype)
    for shape, dtype in zip(init_shapes, init_types)
]
# params = model_mod.load(init_params, f'{CKPT}:img', model_cfg)
params = model_mod.load(init_params, f'{CKPT}:img', model_cfg)

# CapPa

In [ ]:
import importlib

model_cfg = ml_collections.ConfigDict()
model_name = "proj.cappa.cappa"
model_cfg.num_heads = 8
model_cfg.num_layers = 6
model_cfg.mlp_dim = 2048
model_cfg.emb_dim = 512
model_cfg.enc_dropout_rate = 0.
model_cfg.vocab_size = 32_000
model_cfg.seq_len = 256
model_cfg.patches = (16, 16)
model_cfg.input_seq_len = 768
model_cfg.posemb_type = "learn"
model_cfg.patch_dropout = 0.
model_cfg.decoder_num_heads = 0
model_cfg.decoder_num_layers = 0
model_cfg.decoder_mlp_dim = 0
model_cfg.decoder_emb_dim = 0
model_cfg.dec_dropout_rate = 0.
model_cfg.masked_pred_prob = 0.
model_cfg.masking_ratio = 0.
model_cfg.decoder_bias = True
model_cfg.scan = False
model_cfg.remat_policy = "nothing_saveable"
model_cfg.dtype_mm = "float32"

model_mod = importlib.import_module(f"big_vision.models.{model_name}")
model = model_mod.Model(**model_cfg)

CKPT = 'gs://us-central2-storage/tensorflow_datasets/cappa_bs16384_warm0.02_lr1e-3_wd1e-4_bf16_b2-0.95_6lyr_06-15_2102/checkpoint.bv-000183105/'